# Import dataset and Load libraries

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation,Layer,Lambda

In [5]:
data=pd.read_csv('forestfires.csv')
data.head()

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,small


# Data cleaning and visualization

In [7]:
data.shape

(517, 31)

In [10]:
data.drop(['month','day'],axis=1,inplace = True)

In [11]:
data["size_category"].value_counts()

small    378
large    139
Name: size_category, dtype: int64

In [12]:
# taking small as 0 and large as 1
data.loc[data["size_category"]=='small','size_category']=0
data.loc[data["size_category"]=='large','size_category']=1
data["size_category"].value_counts()

0    378
1    139
Name: size_category, dtype: int64

In [13]:
data.head()

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,1,...,0,0,0,0,1,0,0,0,0,0
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,0,...,0,0,0,0,0,0,0,1,0,0
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,0,...,0,0,0,0,0,0,0,1,0,0
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,1,...,0,0,0,0,1,0,0,0,0,0
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,0,...,0,0,0,0,1,0,0,0,0,0


In [14]:
data.shape

(517, 29)

In [15]:
data['size_category']=data['size_category'].astype(int)

In [16]:
data.dtypes

FFMC             float64
DMC              float64
DC               float64
ISI              float64
temp             float64
RH                 int64
wind             float64
rain             float64
area             float64
dayfri             int64
daymon             int64
daysat             int64
daysun             int64
daythu             int64
daytue             int64
daywed             int64
monthapr           int64
monthaug           int64
monthdec           int64
monthfeb           int64
monthjan           int64
monthjul           int64
monthjun           int64
monthmar           int64
monthmay           int64
monthnov           int64
monthoct           int64
monthsep           int64
size_category      int64
dtype: object

In [17]:

#Normalization being done.
def norm_func(i):
     x = (i-i.min()) / (i.max()-i.min())
     return (x)

In [18]:
predictors = data.iloc[:,0:28]
target = data.iloc[:,28]

In [19]:
predictors1 = norm_func(predictors)

In [20]:
predictors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 28 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   FFMC      517 non-null    float64
 1   DMC       517 non-null    float64
 2   DC        517 non-null    float64
 3   ISI       517 non-null    float64
 4   temp      517 non-null    float64
 5   RH        517 non-null    int64  
 6   wind      517 non-null    float64
 7   rain      517 non-null    float64
 8   area      517 non-null    float64
 9   dayfri    517 non-null    int64  
 10  daymon    517 non-null    int64  
 11  daysat    517 non-null    int64  
 12  daysun    517 non-null    int64  
 13  daythu    517 non-null    int64  
 14  daytue    517 non-null    int64  
 15  daywed    517 non-null    int64  
 16  monthapr  517 non-null    int64  
 17  monthaug  517 non-null    int64  
 18  monthdec  517 non-null    int64  
 19  monthfeb  517 non-null    int64  
 20  monthjan  517 non-null    int64 

In [21]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(predictors1,target, test_size=0.3,stratify = target)

In [22]:
def prep_model(hidden_dim):
    model = Sequential()
    for i in range(1,len(hidden_dim)-1):
        if (i==1):
            model.add(Dense(hidden_dim[i],input_dim=hidden_dim[0],activation="relu"))
        else:
            model.add(Dense(hidden_dim[i],activation="relu"))
    model.add(Dense(hidden_dim[-1],kernel_initializer="normal",activation="sigmoid"))
    model.compile(loss="binary_crossentropy",optimizer = "rmsprop",metrics = ["accuracy"])
    return model

In [25]:
first_model = prep_model([28,50,40,20,1])
first_model.fit(np.array(x_train),np.array(y_train),epochs=500)
pred_train = first_model.predict(np.array(x_train))

Epoch 1/500
12/12 [==============================] - 1s 3ms/step - loss: 0.6763 - accuracy: 0.7175
Epoch 2/500
12/12 [==============================] - 0s 2ms/step - loss: 0.6333 - accuracy: 0.7313
Epoch 3/500
12/12 [==============================] - 0s 2ms/step - loss: 0.5967 - accuracy: 0.7313
Epoch 4/500
12/12 [==============================] - 0s 2ms/step - loss: 0.5830 - accuracy: 0.7313
Epoch 5/500
12/12 [==============================] - 0s 2ms/step - loss: 0.5776 - accuracy: 0.7313
Epoch 6/500
12/12 [==============================] - 0s 2ms/step - loss: 0.5768 - accuracy: 0.7313
Epoch 7/500
12/12 [==============================] - 0s 2ms/step - loss: 0.5726 - accuracy: 0.7313
Epoch 8/500
12/12 [==============================] - 0s 2ms/step - loss: 0.5718 - accuracy: 0.7313
Epoch 9/500
12/12 [==============================] - 0s 2ms/step - loss: 0.5707 - accuracy: 0.7313
Epoch 10/500
12/12 [==============================] - 0s 2ms/step - loss: 0.5681 - accuracy: 0.7313
Epoch 11/

In [26]:
pred_train = pd.Series([i[0] for i in pred_train])

In [27]:
size = ["small","large"]
pred_train_class = pd.Series(["small"]*361)
pred_train_class[[i>0.5 for i in pred_train]]= "large"

In [28]:
train = pd.concat([x_train,y_train],axis=1)
train["size_category"].value_counts()

0    264
1     97
Name: size_category, dtype: int64

In [29]:
#For training data
from sklearn.metrics import confusion_matrix
train["original_class"] = "small"
train.loc[train["size_category"]==1,"original_class"] = "large"
train.original_class.value_counts()
confusion_matrix(pred_train_class,train["original_class"])
np.mean(pred_train_class==pd.Series(train["original_class"]).reset_index(drop=True)) #100%
pd.crosstab(pred_train_class,pd.Series(train["original_class"]).reset_index(drop=True))

original_class,large,small
row_0,,
large,96,0
small,1,264


In [30]:
#For test data
pred_test = first_model.predict(np.array(x_test))
pred_test = pd.Series([i[0] for i in pred_test])
pred_test_class = pd.Series(["small"]*156)
pred_test_class[[i>0.5 for i in pred_test]] = "large"
test =pd.concat([x_test,y_test],axis=1)
test["original_class"]="small"
test.loc[test["size_category"]==1,"original_class"] = "large"

5/5 [==============================] - 0s 3ms/step


In [32]:
test["original_class"].value_counts()
np.mean(pred_test_class==pd.Series(test["original_class"]).reset_index(drop=True)) # 85%
confusion_matrix(pred_test_class,test["original_class"])
pd.crosstab(pred_test_class,pd.Series(test["original_class"]).reset_index(drop=True))

original_class,large,small
row_0,,
large,29,5
small,13,109
